In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-storm-10/DATA STORM 1.0 - First Round Competition Guidlines.pdf
/kaggle/input/data-storm-10/credit_card_default_test.csv
/kaggle/input/data-storm-10/credit_card_default_train.csv
/kaggle/input/data-storm-10/Credit_card_default - Business Problem - Assessment Criteria - Data Dictionary.xlsx


In [2]:
df = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_train.csv")
# test_dataset = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_test.csv")

In [3]:
df.head(5)

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,F,High School,Single,31-45,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200K,F,Graduate,Single,31-45,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,F,Graduate,Other,31-45,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [4]:
df.drop(['Client_ID'], 1, inplace=True)
df.fillna(0, inplace=True)

In [5]:
df.sample(5)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
14144,1M,M,Other,Single,31-45,0,0,0,-2,-2,...,0,0,0,2310,0,0,0,0,0,0
3807,400K,M,Other,Other,31-45,1,-2,-2,-1,-1,...,7750,2222,1897,0,0,9300,2222,1897,878,0
204,200K,F,High School,Single,46-65,1,3,2,0,0,...,6805,3403,3403,0,0,0,0,3403,3403,1
4308,100K,M,High School,Single,46-65,0,0,0,2,0,...,29284,30018,31053,3696,4620,0,2310,2310,2310,0
7954,1M,M,Graduate,Other,Less than 30,2,2,2,0,0,...,73959,44010,22747,66309,0,9240,6930,3465,2310,1


In [6]:
df["Balance_Limit_V1"].unique()

array(['1M', '100K', '200K', '400K', ' 500K', '300K', '1.5M', '2.5M'],
      dtype=object)

In [7]:
df["EDUCATION_STATUS"].unique()

array(['Graduate', 'High School', 'Other'], dtype=object)

In [8]:
df["MARITAL_STATUS"].unique()

array(['Other', 'Single'], dtype=object)

In [9]:
df["AGE"].unique()

array(['31-45', 'Less than 30', '46-65', 'More than 65'], dtype=object)

In [10]:
def map_age(age):
    if age == "Less than 30":
        return 0
    elif age == "31-45":
        return 1
    elif age == "46-65":
        return 2
    else:
        return 3

def map_balance(balance):
    if balance == "100K":
        return 1
    elif balance == "200K":
        return 2
    elif balance == "300K":
        return 3
    elif balance == "400K":
        return 4
    elif balance == "500K":
        return 5
    elif balance == "1M":
        return 10
    elif balance == "1.5M":
        return 15
    else:
        return 20


def map_education(education):
    if education == "Other":
        return 0
    elif education == "High School":
        return 1
    elif education == "Graduate":
        return 2

def map_gender(gender):
    if gender == "M":
        return 0
    return 1

def map_marital(marital):
    if marital == "Single":
        return 1
    return 0

In [11]:
df['Balance_Limit_V1'] = df.Balance_Limit_V1.apply(map_balance)
df['EDUCATION_STATUS'] = df.EDUCATION_STATUS.apply(map_education)
df['AGE'] = df.AGE.apply(map_age)
df['Gender'] = df.Gender.apply(map_gender)
df['MARITAL_STATUS'] = df.MARITAL_STATUS.apply(map_marital)

In [12]:
df.sample(10)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
23585,10,1,0,1,1,-1,2,-1,-1,-1,...,9039,1548,5750,0,1264,9039,1548,5750,1515,0
9826,10,1,2,1,1,0,0,0,0,0,...,57713,57711,54865,3250,3077,2225,2026,2310,2097,1
16630,10,1,0,1,2,0,0,0,0,0,...,602716,593672,485176,39270,41580,30030,27720,27720,18480,0
16983,4,1,2,1,1,-1,-1,-1,-1,-1,...,7785,16814,10268,6972,14324,7822,16861,2693,12,0
10568,2,1,0,0,1,0,0,0,0,0,...,43072,44313,44384,6999,4906,1492,1913,2534,1414,0
22606,2,0,0,1,1,0,0,0,0,0,...,117505,27727,22358,4179,4518,5161,852,762,792,0
5477,10,0,2,1,1,0,0,0,0,0,...,462270,471961,481342,22266,16454,16544,17138,17237,19915,0
15067,20,0,2,0,0,0,0,0,0,0,...,263735,267445,274310,9240,13860,13860,11550,11550,26565,0
12648,2,0,0,0,0,0,0,0,0,0,...,149533,154031,164571,6930,6930,5775,6930,13398,0,0
17676,3,0,0,0,0,0,0,0,-1,-1,...,5036,4502,1656,11550,2310,5036,4502,1656,90090,0


In [13]:
from sklearn import preprocessing
X = np.array(df.drop(['NEXT_MONTH_DEFAULT'], 1).astype(float))
# X = preprocessing.scale(X)
y = np.array(df['NEXT_MONTH_DEFAULT'])

In [14]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [15]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=2)
clf.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [16]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

0.6983333333333334
